In [1]:
with open('LSTMtext.txt', 'r') as file:
    doc = file.read()    

In [2]:
from keras.preprocessing.text import Tokenizer

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([doc])
tokenizer.word_index

{'the': 1,
 'a': 2,
 'is': 3,
 'cricket': 4,
 'in': 5,
 'what': 6,
 'to': 7,
 'of': 8,
 'ball': 9,
 'and': 10,
 'by': 11,
 'or': 12,
 'batsman': 13,
 'team': 14,
 'with': 15,
 'on': 16,
 'an': 17,
 'for': 18,
 'leg': 19,
 'shot': 20,
 'batting': 21,
 'bowler': 22,
 'bat': 23,
 'are': 24,
 'runs': 25,
 'delivery': 26,
 'that': 27,
 'match': 28,
 'overs': 29,
 'role': 30,
 'bowled': 31,
 'played': 32,
 'test': 33,
 'batsmen': 34,
 'off': 35,
 'side': 36,
 'when': 37,
 'between': 38,
 'matches': 39,
 'as': 40,
 'run': 41,
 'often': 42,
 'field': 43,
 'innings': 44,
 'where': 45,
 'century': 46,
 'used': 47,
 "batsman's": 48,
 'fielding': 49,
 'towards': 50,
 'refers': 51,
 'pitch': 52,
 'over': 53,
 'making': 54,
 'stroke': 55,
 'each': 56,
 'international': 57,
 'typically': 58,
 'out': 59,
 'from': 60,
 'their': 61,
 'limited': 62,
 'body': 63,
 'umpire': 64,
 'spinner': 65,
 'teams': 66,
 'which': 67,
 'such': 68,
 'stumps': 69,
 'more': 70,
 'lbw': 71,
 'first': 72,
 'fast': 73,
 'dec

In [4]:
# sentences = doc.split('\n')
Uni_Gram = []
Bi_Gram = []
Tri_Gram = []
Quad_Gram = []
Penta_Gram = []
Hexa_Gram = []

for sentence in doc.split('\n'):
    tokenized_sentences = tokenizer.texts_to_sequences([sentence])[0]
    
    for i in range(1, len(tokenized_sentences)):
        Uni_Gram.append(tokenized_sentences[:i])
        Bi_Gram.append(tokenized_sentences[:i+1])
        Tri_Gram.append(tokenized_sentences[:i+2])
        Quad_Gram.append(tokenized_sentences[:i+3])
        Penta_Gram.append(tokenized_sentences[:i+4])
        Hexa_Gram.append(tokenized_sentences[:i+5])
    

In [5]:

maxLen = max([len(x) for x in Bi_Gram])
maxLen

58

In [6]:
from keras.preprocessing.sequence import pad_sequences
padded_uni_gram_sequences = pad_sequences(Uni_Gram, maxlen=maxLen, padding='pre')
padded_bi_gram_sequences = pad_sequences(Bi_Gram, maxlen=maxLen, padding='pre')
padded_tri_gram_sequences = pad_sequences(Tri_Gram, maxlen=maxLen, padding='pre')
padded_quad_gram_sequences = pad_sequences(Quad_Gram, maxlen=maxLen, padding='pre')
padded_penta_gram_sequences = pad_sequences(Penta_Gram, maxlen=maxLen, padding='pre')
padded_hexa_gram_sequences = pad_sequences(Hexa_Gram, maxlen=maxLen, padding='pre')

In [7]:
# print(padded_uni_gram_sequences)
# print(padded_uni_gram_sequences[:,:-1][1])
X_uni_gram = padded_uni_gram_sequences[:,:-1]
X_bi_gram = padded_bi_gram_sequences[:,:-1]
X_tri_gram = padded_tri_gram_sequences[:,:-1]
X_quad_gram = padded_quad_gram_sequences[:,:-1]
X_penta_gram = padded_penta_gram_sequences[:,:-1]
X_hexa_gram = padded_hexa_gram_sequences[:,:-1]

y_uni_gram = padded_uni_gram_sequences[:,-1]
y_bi_gram = padded_bi_gram_sequences[:,-1]
y_tri_gram = padded_tri_gram_sequences[:,-1]
y_quad_gram = padded_quad_gram_sequences[:,-1]
y_penta_gram = padded_penta_gram_sequences[:,-1]
y_hexa_gram = padded_hexa_gram_sequences[:,-1]

print(X_uni_gram.shape, X_bi_gram.shape, X_tri_gram.shape, X_quad_gram.shape, X_penta_gram.shape, X_hexa_gram.shape)
print(y_uni_gram.shape, y_bi_gram.shape, y_tri_gram.shape, y_quad_gram.shape, y_penta_gram.shape, y_hexa_gram.shape)

(3421, 57) (3421, 57) (3421, 57) (3421, 57) (3421, 57) (3421, 57)
(3421,) (3421,) (3421,) (3421,) (3421,) (3421,)


In [8]:
from keras.utils import to_categorical

In [9]:
y_uni_gram = to_categorical(y_uni_gram, num_classes=len(tokenizer.word_index)+1)

In [10]:
y_bi_gram = to_categorical(y_bi_gram, num_classes=len(tokenizer.word_index)+1)

In [11]:
y_tri_gram = to_categorical(y_tri_gram, num_classes=len(tokenizer.word_index)+1)

In [12]:
y_quad_gram = to_categorical(y_quad_gram, num_classes=len(tokenizer.word_index)+1)

In [13]:
y_penta_gram = to_categorical(y_penta_gram, num_classes=len(tokenizer.word_index)+1)

In [14]:
y_hexa_gram = to_categorical(y_hexa_gram, num_classes=len(tokenizer.word_index)+1)

In [15]:
print(y_uni_gram.shape, y_bi_gram.shape, y_tri_gram.shape, y_quad_gram.shape, y_penta_gram.shape, y_hexa_gram.shape)

(3421, 810) (3421, 810) (3421, 810) (3421, 810) (3421, 810) (3421, 810)


In [16]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, GRU

In [17]:
vocab_size = len(tokenizer.word_index)
padded_sequence_Length = len(padded_bi_gram_sequences[0])
print(vocab_size, padded_sequence_Length)

809 58


In [18]:
# model = Sequential()
# model.add(Embedding(input_dim=vocab_size+1, output_dim=100, input_length=padded_sequence_Length-1))
# model.add(LSTM(units=150))
# model.add(Dense(units=vocab_size+1, activation='softmax'))
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

model = Sequential()
model.add(Embedding(input_dim=vocab_size+1, output_dim=100, input_length=padded_sequence_Length-1))
model.add(GRU(units=150))
model.add(Dense(units=vocab_size+1, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 57, 100)           81000     
                                                                 
 gru (GRU)                   (None, 150)               113400    
                                                                 
 dense (Dense)               (None, 810)               122310    
                                                                 
Total params: 316710 (1.21 MB)
Trainable params: 316710 (1.21 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
history = model.fit(X_bi_gram, y_bi_gram, epochs=100, verbose=1)

Epoch 1/100


107/107 [==============================] - 11s 61ms/step - loss: 5.6951 - accuracy: 0.0778
Epoch 2/100
107/107 [==============================] - 7s 64ms/step - loss: 4.9074 - accuracy: 0.1713
Epoch 3/100
107/107 [==============================] - 5s 44ms/step - loss: 4.3929 - accuracy: 0.2283
Epoch 4/100
107/107 [==============================] - 6s 52ms/step - loss: 3.9846 - accuracy: 0.2640
Epoch 5/100
107/107 [==============================] - 6s 59ms/step - loss: 3.6281 - accuracy: 0.3128
Epoch 6/100
107/107 [==============================] - 7s 64ms/step - loss: 3.3105 - accuracy: 0.3455
Epoch 7/100
107/107 [==============================] - 6s 52ms/step - loss: 3.0149 - accuracy: 0.3803
Epoch 8/100
107/107 [==============================] - 5s 50ms/step - loss: 2.7476 - accuracy: 0.4104
Epoch 9/100
107/107 [==============================] - 6s 54ms/step - loss: 2.4887 - accuracy: 0.4467
Epoch 10/100
107/107 [==============================] - 6s 52ms/step - loss: 2.

In [20]:
import numpy as np

In [21]:
import time
text = "The wicket-keeper's role is to"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=padded_sequence_Length-1, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)

1/1 [==============================] - 1s 614ms/step
The wicket-keeper's role is to stand
1/1 [==============================] - 0s 24ms/step
The wicket-keeper's role is to stand behind
1/1 [==============================] - 0s 25ms/step
The wicket-keeper's role is to stand behind the
1/1 [==============================] - 0s 26ms/step
The wicket-keeper's role is to stand behind the stumps
1/1 [==============================] - 0s 21ms/step
The wicket-keeper's role is to stand behind the stumps to
1/1 [==============================] - 0s 24ms/step
The wicket-keeper's role is to stand behind the stumps to catch
1/1 [==============================] - 0s 23ms/step
The wicket-keeper's role is to stand behind the stumps to catch balls
1/1 [==============================] - 0s 25ms/step
The wicket-keeper's role is to stand behind the stumps to catch balls that
1/1 [==============================] - 0s 27ms/step
The wicket-keeper's role is to stand behind the stumps to catch balls that the
1

In [64]:
model.loss

'categorical_crossentropy'

In [88]:
# Uni_Gram = []
# Bi_Gram = []
# Tri_Gram = []
# Quad_Gram = []
# Penta_Gram = []
# Hexa_Gram = []

# tokenized_sentences = tokenizer.texts_to_sequences(sentences)[0]
# tokenized_sentences

# # for i in tokenized_sentences:
# #     for i in range(1,len(tokenized_sentences)):
# #         Uni_Gram.append(tokenized_sentences[:i])
# #         Bi_Gram.append(tokenized_sentences[:i+1])
        
# # Bi_Gram


[351, 6, 3, 4]